In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 载入数据
mnist = input_data.read_data_sets('MNIST_DATA',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_DATA/t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
#一张28*28的图片,每一行28个像素点,总共28行,每次输入一行,总共输入28次
n_inputs = 28 # 输入1行,一行28个数据
max_time = 28 # 总共输入28次
lstm_size = 100 # 隐藏层单元
n_classes = 10
batch_size = 100
n_batch = mnist.train.num_examples // batch_size

x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None,10])

weights = tf.Variable(tf.truncated_normal([lstm_size, n_classes], stddev=0.1))
biases = tf.Variable(tf.constant(0.1, shape=[n_classes]))

# 定义RNN
def RNN(X, weight, biases):
    # inputs = [batch_size, max_time(序列长度), n_inputs(序列中,每次传入数据大小)]
    inputs = tf.reshape(X, [-1,max_time,n_inputs])
    # 定义LSTM基本cell
    #lstm_cell = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(lstm_size)
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    # state[0]:cell state
    # state[1]:hidden_state
    outputs,final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)
    results = tf.nn.softmax(tf.matmul(final_state[1], weights) + biases)
    return results

# 计算RNN返回结果
prediction = RNN(x, weights, biases)
cross_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_loss)
cross_predictoin = tf.equal(tf.math.argmax(prediction, 1), tf.math.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(cross_predictoin, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
            
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('iter '+ str(epoch) + ', accuracy: ' + str(acc))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
iter 0, accuracy: 0.6098
iter 1, accuracy: 0.7627
iter 2, accuracy: 0.794
iter 3, accuracy: 0.8803
iter 4, accuracy: 0.9063
iter 5, accuracy: 0.9091
